In [4]:
from google.colab import drive
import os
import zipfile
import requests

drive.mount('/content/drive')

images_path = "/content/drive/MyDrive/images.tar"

dataset_url = "http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar"

res = requests.get(url=dataset_url)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
with open(images_path,mode='wb') as f:  
    f.write(res.content)

In [7]:
import tarfile
def un_tar(file_name):  
      tar = tarfile.open(file_name)  
      names = tar.getnames()  
      if os.path.isdir(file_name + "_files"):  
          pass  
      else:  
          os.mkdir(file_name + "_files")  
      for name in names:  
          tar.extract(name, file_name + "_files/")  
      tar.close()  
un_tar(images_path)


In [8]:
classification_url = "http://vision.stanford.edu/aditya86/ImageNetDogs/lists.tar"

classification_path = "/content/drive/MyDrive/lists.tar"

classification_res = requests.get(url=classification_url)

with open(classification_path,mode='wb') as f:  
    f.write(classification_res.content)

In [9]:
un_tar(classification_path)

In [10]:
import scipy.io as scio
import shutil

# load the test and train classification list
test_path = "/content/drive/MyDrive/lists.tar_files/test_list.mat"
train_path = "/content/drive/MyDrive/lists.tar_files/train_list.mat"

# images path (untar by images.tar)
path_img = "/content/drive/MyDrive/images.tar_files/"
if not os.path.exists(path_img+"test"):
    os.makedirs(path_img+"test")
if not os.path.exists(path_img+"train"):
    os.makedirs(path_img+"train")

test_data = scio.loadmat(test_path)

# create test folder
for t in test_data["labels"]:
    if not os.path.exists(path_img+"test/"+str(t[0])):
        os.makedirs(path_img+"test/"+str(t[0]))
train_data = scio.loadmat(train_path)

# create train folder
for t in train_data["labels"]:
    if not os.path.exists(path_img+"train/"+str(t[0])):
        os.makedirs(path_img+"train/"+str(t[0]))

# classify the test images to test folder
for t in range(len(test_data["file_list"])):
    source_path = (path_img+"Images/"+test_data["file_list"][t][0][0])
    target_path = (path_img+"test/"+str(test_data["labels"][t][0])+"/"+test_data["file_list"][t][0][0].split("/", 1)[1])
    shutil.copyfile(source_path,target_path)

# classify the train images to test folder
for t in range(len(train_data["file_list"])):
    source_path = (path_img+"Images/"+train_data["file_list"][t][0][0])
    target_path = (path_img+"train/"+str(train_data["labels"][t][0])+"/"+train_data["file_list"][t][0][0].split("/", 1)[1])
    shutil.copyfile(source_path,target_path)

In [11]:
import pandas as pd
import numpy as np
from numpy import expand_dims
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.transform import resize
from IPython.display import SVG
from tensorflow import keras
from keras import applications
from keras import optimizers
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.utils.np_utils import to_categorical
from keras import layers, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau

test_path = "/content/drive/MyDrive/images.tar_files/test"
train_path = "/content/drive/MyDrive/images.tar_files/train"

img_width, img_height = 224, 224
channels = 3
batch_size = 64
num_images = 100
image_arr_size = img_width * img_height * channels

In [12]:
train_datagen = ImageDataGenerator(
    rescale= 1./255,
    shear_range= 0.2,
    zoom_range= 0.2,
    horizontal_flip= True,
    rotation_range= 20,
    width_shift_range= 0.2,
    height_shift_range= 0.2,
    validation_split=0.2,

)

valid_datagen = ImageDataGenerator(
    rescale= 1./255,
    validation_split=0.2,
)


train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size= (img_width, img_height),
    color_mode= 'rgb',
    batch_size= batch_size,
    class_mode= 'categorical',
    subset='training',
    shuffle= True,
    seed= 1337
)

valid_generator = valid_datagen.flow_from_directory(
    train_path,
    target_size= (img_width, img_height),
    color_mode= 'rgb',
    batch_size= batch_size,
    class_mode= 'categorical',
    subset='validation',
    shuffle= True,
    seed= 1337
)

Found 9600 images belonging to 120 classes.
Found 2400 images belonging to 120 classes.


In [13]:
num_classes = len(train_generator.class_indices)
train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)
valid_labels = valid_generator.classes
valid_labels = to_categorical(valid_labels, num_classes=num_classes)
nb_train_samples = len(train_generator.filenames)
nb_valid_samples = len(valid_generator.filenames)

In [14]:
test_datagen = ImageDataGenerator(rescale=1/255.0)
test_generator = test_datagen.flow_from_directory(
                                    test_path,
                                    target_size=(img_width, img_height),
                                    color_mode= 'rgb',
                                    batch_size=batch_size,
                                    class_mode= 'categorical',
                                    shuffle=True,)
test_generator.reset()

Found 8580 images belonging to 120 classes.
